<p style="font-size:300%; background-color:pink; color:blue; text-align:center;line-height : 80px; margin : 0; padding : 0;">
Taller 5</p>
<p style="font-size:240%; background-color:pink; color:red; text-align:center;line-height : 60px; margin : 0; padding : 0;">
Técnicas de Inteligencia Artificial - Visión de Máquina</p1>

<p style="font-size:200%; text-align:center; line-height : 40px;  margin-top : 0; margin-bottom : 0; "> <br>Jose Eduardo Garnica Aza - Johan Sebastian Suarez Sepulveda</p>
<p style="font-size:160%; text-align:center; line-height : 20px; margin-top : 0; "> email: jgarnicaa@unal.edu.co email: jssuarezse@unal.edu.co</p>
<p style="font-size:180%; text-align:center; line-height : 30px;  margin-top : 0; margin-bottom : 0; "> <br><br>INGENIERÍA MECATRÓNICA</p>
<p style="font-size:180%; text-align:center; line-height : 30px; margin-top : 0; "> Facultad de Ingeniería</p>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> Universidad Nacional de Colombia Sede Bogotá</p>
<br>
<img src="Integrantes.png",width=10,height=10>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> <br>4 de mayo de 2022</p>

# Extracción de caracteristicas
El objetivo de este taller es tratar 30 imagenes concernientes al proyecto y definir un vector de caracteristicas mayor o igual a 30. 

### Plan de acción
- Definición de caracteristicas
- Carga de imagenes
- Preproceso de imagenes
- Hallar momentos 
- Realizar análisis de componentes principales

In [1]:
#librerias
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap
import numpy as np 
import math as m
import time
import timeit as tm
import cv2 as cv
import pandas as pd
from pylab import plot,subplot,axis,stem,show,figure
from numpy import mean,cov,double,cumsum,dot,linalg,array
import os
from scipy import signal
from scipy import misc

from sklearn import neighbors, datasets

In [2]:
#codigo para importar grupo de imagenes

inputpath="InputImages"
inputnames= os.listdir(inputpath)
print(inputnames)

outputpath="OutputImages"
if not os.path.exists(outputpath):
    os.makedirs(outputpath)
    print("Directorio de salida creado")

sift = cv.SIFT_create()

['batman1.jpg', 'batman10.jpg', 'batman2.jpg', 'batman3.jpg', 'batman4.jpg', 'batman5.jpg', 'batman6.jpg', 'batman7.jpg', 'batman8.png', 'batman9.jpg', 'flash1.jpg', 'flash10.jpg', 'flash2.jpg', 'flash3.jpg', 'flash4.png', 'flash5.png', 'flash6.png', 'flash7.jpg', 'flash8.jpg', 'flash9.jpg', 'superman1.png', 'superman10.jpg', 'superman2.jpg', 'superman3.jpg', 'superman4.jpg', 'superman5.jpg', 'superman6.jpg', 'superman7.jpg', 'Superman8.jpg', 'superman9.jpg']


In [3]:
#codigo para lectura y proceso de imagenes (canny)
#trabajar siempre dentro del for para operar todas las imagenes al tiempo

#Cada columna de la matriz representa una imagen y cada fila una caracteristica

M=np.zeros((40,30))


count =0
for inputnames in inputnames:

    #---------------------Pre proceso de imagenes--------------------------------#

    imagepath=inputpath+"/"+inputnames
    image=cv.imread(imagepath)
    imagegray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    canny = cv.Canny(image , 25, 50, apertureSize = 3)
    cv.imwrite(outputpath+"/imagecan"+str(count)+".jpg", canny)
    #Detecte puntos clave
    kp, des = sift.detectAndCompute(canny, None)
    Keypoints = cv.drawKeypoints(canny,kp,canny,color=(255,0,255)) # Dibuje los puntos característicos y muéstrelos como círculos rojos
    cv.imwrite(outputpath+"/keypoints"+str(count)+".jpg", Keypoints)
    #Hallar contornos
    ret, thresh = cv.threshold(imagegray, 127, 255, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    img_contours = np.zeros(image.shape)
    cv.drawContours(img_contours, contours, -1, (0,255,0), 3)
    cv.imwrite(outputpath+"/contours"+str(count)+".jpg", img_contours)
    imgmedian= cv.medianBlur(imagegray,5)
    

    #-----------------------Extracción de caracteristicas-------------------------#    
    cnt = contours[0]


    Rx,Ry,Rw,Rh=cv.boundingRect(cnt)
    RectM=cv.minAreaRect(cnt)
    box = cv.boxPoints(RectM)
    BaseRectM=m.sqrt(abs(pow(((box[1][0])-box[0][0]),2)-pow(((box[1][1])-box[0][1]),2)))
    AlturaRectM=m.sqrt(abs(pow(((box[2][0])-box[1][0]),2)-pow(((box[2][1])-box[1][1]),2)))

    (Cx,Cy),radius = cv.minEnclosingCircle(cnt)
    linesH = cv.HoughLines(canny, 1, np.pi / 180, 200)
    linesHP = cv.HoughLinesP(canny,1,np.pi/180,100,minLineLength=100,maxLineGap=10)
    
    CirclesH= cv.HoughCircles(imgmedian,cv.HOUGH_GRADIENT,1,120,param1=100,param2=30,minRadius=0,maxRadius=0)
    CirclesH= np.uint16(np.around(CirclesH))

    Momentos=cv.moments(cnt)
    MomentosHu = cv.HuMoments(Momentos)


    PerimetroContorno = cv.arcLength(cnt, 1)
    AreaContorno = cv.contourArea(cnt)
    PerimetroRect=2*Rw+2*Rh
    AreaRect=Rw*Rh
    PerimetroRectM=2*BaseRectM+2*AlturaRectM
    AreaRectM=BaseRectM*AlturaRectM
    radius = int(radius)
    AreaCir= m.pi*radius**2
    PerimetroCir=2*m.pi*radius
    NLinesH=np.size(linesH)
    NLinesHP=np.size(linesHP)
    NCirculosH=np.size(CirclesH[0,:])

    #-----------------------Llenado de caracteristicas en la matriz---------------#

    M[0][count]=PerimetroContorno #Primera fila perimetro contornos
    M[1][count]=AreaContorno #Segunda fila area de contornos
    M[2][count]=PerimetroRect #Cuarta fila perimetro rectangulo 
    M[3][count]=AreaRect #Quinta fila Area rectangulo
    M[4][count]=PerimetroRectM #Sexta fila perimetro rectagulo minimo
    M[5][count]=AreaRectM #Septima fila area rectagulo minimo
    M[6][count]=PerimetroCir #Octava fila perimetro del circulo
    M[7][count]=AreaCir #Novena fila area circulo
    M[8][count]=NCirculosH #Fila 12 numero de circulos Hough
    M[9][count]= Momentos['m00']#Filas para momentos espaciales
    M[10][count]= Momentos['m10']
    M[11][count]= Momentos['m01']
    M[12][count]= Momentos['m20']
    M[13][count]= Momentos['m11']
    M[14][count]= Momentos['m02']
    M[15][count]= Momentos['m30']
    M[16][count]= Momentos['m21']
    M[17][count]= Momentos['m12']
    M[18][count]= Momentos['m03']
    M[19][count]=Momentos['mu20'] #Filas para momentos centralizados
    M[20][count]=Momentos['mu11']
    M[21][count]=Momentos['mu02']
    M[22][count]=Momentos['mu30']
    M[23][count]=Momentos['mu21']
    M[24][count]=Momentos['mu12']
    M[25][count]=Momentos['mu03']
    M[26][count]=Momentos['nu20']#Filas para momentos centralizados normalizados
    M[27][count]=Momentos['nu11']
    M[28][count]=Momentos['nu02']
    M[29][count]=Momentos['nu30']
    M[30][count]=Momentos['nu21']
    M[31][count]=Momentos['nu12']
    M[32][count]=Momentos['nu03']
    M[33][count]=MomentosHu[0]
    M[34][count]=MomentosHu[1]
    M[35][count]=MomentosHu[2]
    M[36][count]=MomentosHu[3]
    M[37][count]=MomentosHu[4]
    M[38][count]=MomentosHu[5]
    M[39][count]=MomentosHu[6]
    count +=1

#Matriz traspuesta para que las caracteristicas queden en modo columna
Mt=np.transpose(M)


# Para el análisis de componentes principales utilizaremos la funcion PCA de openCv

In [4]:
mean = np.empty((0))
mean, eigvector  = cv.PCACompute(Mt.astype(double), mean)
print ("\nMedia")
print (mean)
print ("\nEigenVectors") # Mostrados en Filas
print (eigvector)
variabilidad = 0
print ("\nValores propios")
MC = np.cov(Mt.transpose())  
for i in range (eigvector.shape[0]):
    eigenValue = np.dot(np.dot(eigvector[i].transpose(), MC), eigvector[i])/ np.dot(eigvector[i].transpose(), eigvector[i])
    variabilidad= variabilidad+ eigenValue
    print (eigenValue)


Media
[[ 1.10602934e+03  1.75794800e+05  1.11693333e+03  1.88687933e+05
   1.07040509e+03  1.82462566e+05  1.19673736e+03  2.90241902e+05
   5.46000000e+01  1.75794800e+05  7.49449425e+07  8.42597304e+07
   4.73407082e+10  4.10637144e+10  6.24691464e+10  3.63427639e+13
   2.85361082e+13  3.33290478e+13  5.86664119e+13  1.15372396e+10
  -4.19320031e+06  1.47425212e+10 -2.60885425e+08 -5.33646492e+08
  -2.66167166e+08 -3.57249341e+08  9.45871613e-02 -3.67085824e-02
   1.17138609e-01 -3.50949722e-02  2.38486092e-02 -6.41969161e-03
  -5.05781115e-02  2.11725771e-01  1.68813054e-01  4.74030212e-01
   7.20710871e-02 -3.99241879e-01 -1.61179606e-01 -9.65621777e-03]]

EigenVectors
[[ 2.56301681e-12  9.17218456e-10  2.57684348e-12 ...  2.69551924e-16
   1.08821966e-16  6.51948649e-18]
 [ 5.14689077e-11  9.36578484e-09  5.56683582e-11 ...  1.05788944e-14
   4.27085818e-15  2.55865190e-16]
 [ 1.82193117e-10  3.79915502e-08  1.35711333e-10 ...  4.56492835e-14
   1.84291948e-14  1.10409133e-15]
 .

# La variabilidad será igual a la suma de todos los valores propios

In [5]:
print("variabilidad = ")
print(variabilidad)

variabilidad = 
1.2238744327607701e+29


# Ya obtuvimos la variabilidad del conjunto de datos, quitaremos las primeras 15 caracteristicas y realizaremos el mismo procedimiento

In [6]:
Mt2=np.delete(Mt,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)




In [7]:
mean = np.empty((0))
mean, eigvector  = cv.PCACompute(Mt2.astype(double), mean)
variabilidad2 = 0
MC = np.cov(Mt2.transpose())  
for i in range (eigvector.shape[0]):
    eigenValue = np.dot(np.dot(eigvector[i].transpose(), MC), eigvector[i])/ np.dot(eigvector[i].transpose(), eigvector[i])
    variabilidad2= variabilidad2+ eigenValue
print("variabilidad = ")
print(variabilidad2)
print("Porcentaje de variabilidad = ")
print((variabilidad2*100)/variabilidad)

variabilidad = 
1.0273430329111825e+29
Porcentaje de variabilidad = 
83.9418657185068


- En la caracteristica #16, el porcentaje de la variabilidad queda en 89%, sin embargo al mirar las demás caracteristicas, no afectan demasiado la variabilidad, por tanto se descartarán.

In [8]:
Mt2=np.delete(Mt,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0, axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,0,axis=1)
Mt2=np.delete(Mt2,4,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,6,axis=1)
Mt2=np.delete(Mt2,5,axis=1)
Mt2=np.delete(Mt2,4,axis=1)
mean = np.empty((0))
mean, eigvector  = cv.PCACompute(Mt2.astype(double), mean)
variabilidad2 = 0
MC = np.cov(Mt2.transpose())  
for i in range (eigvector.shape[0]):
    eigenValue = np.dot(np.dot(eigvector[i].transpose(), MC), eigvector[i])/ np.dot(eigvector[i].transpose(), eigvector[i])
    variabilidad2= variabilidad2+ eigenValue
print("variabilidad = ")
print(variabilidad2)
print("Porcentaje de variabilidad = ")
print((variabilidad2*100)/variabilidad)

#Guardar array para taller 6 
np.savetxt('Mt2.txt',Mt2)

variabilidad = 
1.2238733597232122e+29
Porcentaje de variabilidad = 
99.99991232453843


- De las 41 caraterísticas, 5 características nos dan en total la variabilidad, por lo cual estas 5 serian las optimas a utilizar

## Referencias bibliograficas y solución de dudas
- https://stats.stackexchange.com/questions/22569/pca-and-proportion-of-variance-explained
- https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/
- https://www.pluralsight.com/guides/building-features-from-image-data-in-python
